<a href="https://colab.research.google.com/github/Tushar-Shinde31/Blood_Group_Detection_Using_Fingerprint/blob/main/magic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/My_ML_Data/test'
val_dir = '/content/drive/MyDrive/My_ML_Data/train'
test_dir = '/content/drive/MyDrive/My_ML_Data/val'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [ ]:
# Load dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 924 images belonging to 8 classes.
Found 4305 images belonging to 8 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers for transfer learning
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(8, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
checkpoint_path = "/content/drive/MyDrive/Docs/best_model.keras"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)


In [ ]:
# EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.1400 - loss: 2.1831
Epoch 1: val_accuracy improved from -inf to 0.17677, saving model to /content/drive/MyDrive/Docs/best_model.keras
29/29 ━━━━━━━━━━━━━━━━━━━━ 1674s 57s/step - accuracy: 0.1400 - loss: 2.1816 - val_accuracy: 0.1768 - val_loss: 2.0780
Epoch 2/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step - accuracy: 0.1755 - loss: 2.0672
Epoch 2: val_accuracy did not improve from 0.17677
29/29 ━━━━━━━━━━━━━━━━━━━━ 31s 982ms/step - accuracy: 0.1751 - loss: 2.0673 - val_accuracy: 0.1751 - val_loss: 2.0636
Epoch 3/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.1972 - loss: 2.0306
Epoch 3: val_accuracy did not improve from 0.17677
29/29 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.1967 - loss: 2.0309 - val_accuracy: 0.1524 - val_loss: 2.0451
Epoch 4/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - accuracy: 0.1561 - loss: 2.0332
Epoch 4: val_accuracy improved from 0.17677 to 0.20999, saving model to /content/drive/MyDrive/Docs/bes

In [ ]:
final_model_path = "/content/drive/MyDrive/Docs/final_blood_group_model.keras"
model.save(final_model_path)
print("Model saved at:", final_model_path)

Model saved at: /content/drive/MyDrive/Docs/final_blood_group_model.keras
